# 1.将text划分为token

In [8]:
'''Load data'''
import os
import urllib.request

if not os.path.exists("./data/the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "./data/the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

with open("./data/the-verdict.txt", 'r', encoding = 'utf-8') as f:
    raw_text = f.read()

print(len(raw_text))
print(raw_text[0:99])

20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [9]:
'''Process the text using regular expression'''
import re
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

print(preprocessed[0:30])
print(len(preprocessed)) #这就是token的数量

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']
4690


# 2.将Token映射为Token IDs 

In [10]:
'''Build vocabulary containing all these tokens'''
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size) 

# Note
# 构建字典的时候，需要先去重，然后按照字母表的顺序排序

1130


In [11]:
vocab = {token:integer for integer,token in enumerate(all_words)}

for i, item in enumerate(vocab.items()):
    print(item)
    if i == 20:
        break

# Note:
# token作为字典的key的原因是，现在是token -> token IDs，是拿token去查

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)


In [12]:
'''Put all these operations into a tokenizer class'''
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab #词典是提前建立好的，tokenizer只是拿过来用了
        self.int_to_str = {i:s for s, i in vocab.items()}
    
    def encode(self, text):
        '''将token转换为token ID'''
        # 查词典之前需要先对文本进行处理
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]

        #在vocab中取出token对应的token ID
        ids = [self.str_to_int[s] for s in preprocessed]

        return ids

    def decode(self, ids):
        '''将token ID转换为文本'''
        text = " ".join([self.int_to_str[i] for i in ids])

        # 将特殊字符前的空格替换
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)

        return text

In [13]:
'''Tokenize text using the tokenizer'''

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""

tokenizer = SimpleTokenizerV1(vocab)
ids = tokenizer.encode(text)

print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [14]:
'''Decode the integers back into text'''
print(tokenizer.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


# 3.添加特殊的context tokens


In [15]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text) #Hello在字典中是没有的

KeyError: 'Hello'

In [ ]:
'''Adding context tokens <|endoftext|> and <|unk|> into the vocabulary'''
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer, token in enumerate(all_tokens)}

In [ ]:
print(len(vocab))

for i,item in enumerate(list(vocab.items())[-5:]):
    print(item)

1132
('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [ ]:
'''Replace unknown words in the input text with <|unk|> token'''
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab #词典是提前建立好的，tokenizer只是拿过来用了
        self.int_to_str = {i:s for s, i in vocab.items()}
    
    def encode(self, text):
        '''将token转换为token ID'''
        # 查词典之前需要先对文本进行处理
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        
        # 将输入中，在字典中没有的词，替换为<|unk|>
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        #在vocab中取出token对应的token ID
        ids = [self.str_to_int[s] for s in preprocessed]

        return ids

    def decode(self, ids):
        '''将token ID转换为文本'''
        text = " ".join([self.int_to_str[i] for i in ids])

        # 将特殊字符前的空格替换
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)

        return text

In [ ]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

# 使用<|endoftext|>连接不相关的文本，注意在连接的时候，前后一定要加空格！！！
text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [ ]:
# 对包含未知单词的文本进行tokenization
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [ ]:
# 对tokenized的结果再de-tokenization
print(tokenizer.decode(tokenizer.encode(text)))

# 可以发现，在de-tokenization的时候，如果是存在字典中未知的token，就会无法解析回到原始的文本了

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


# 4.字节对编码BPE

In [ ]:
import importlib
import tiktoken

print(importlib.metadata.version("tiktoken"))

0.9.0


In [ ]:
'''Load the tokenizer'''
tokenizer = tiktoken.get_encoding("gpt2")

# Note：
# gpt2的tokenizer词典一共50257个词

In [ ]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

'''Tokenization'''
integers = tokenizer.encode(text, allowed_special = {"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [ ]:
'''De-tokenization'''
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


# 5.使用滑动窗口进行数据采样 

In [4]:
'''Load data and encode it using tokenizer'''
import tiktoken

with open('./data/the-verdict.txt', 'r', encoding = "utf-8") as f:
    raw_text = f.read()

tokenizer = tiktoken.get_encoding("gpt2")
enc_text = tokenizer.encode(raw_text)

print(type(enc_text))
print(len(enc_text))

<class 'list'>
5145


In [5]:
enc_sample = enc_text[50 : ] # 只是为了和教程中的保持一致

In [ ]:
context_size = 4

# 为了实现next token prediction任务，那么y中就要包含x中不存在的下一个token
# 但是y是x向后移动了一位的结果，那么第一位的token就被y忽略了，这个是为什么呢，还是说实际训练
# 的过程中，只会使用y的末位，也就是x的下一个token，所以前面忽略了也无所谓？
x = enc_sample[:context_size]
y = enc_sample[1:context_size + 1]

print(enc_sample[0:10])
print(x)
print(y)

[290, 4920, 2241, 287, 257, 4489, 64, 319, 262, 34686]
[290, 4920, 2241, 287]
[4920, 2241, 287, 257]


In [ ]:
'''Simulate the next token prediction task'''
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print("{}-->{}".format(context, desired))


for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print("{}-->{}".format(tokenizer.decode(context), tokenizer.decode([desired])))

# Note:
# 1.tokenizer.decode()接收的参数不能是一个数值类型，
# 而是一个sequence类型（一个包含token IDs的list）

[290]-->4920
[290, 4920]-->2241
[290, 4920, 2241]-->287
[290, 4920, 2241, 287]-->257
 and--> established
 and established--> himself
 and established himself--> in
 and established himself in--> a


In [14]:
'''Implment a dataloader to get the input-target pair batch'''
import torch
print(torch.__version__)
from torch.utils.data import Dataset, DataLoader

2.4.1+cu118


In [42]:
# Create the dataset
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []  # 输入的token序列
        self.target_ids = [] # 目标的token序列，包含了要预测的下一个token

        # Tokenize the entire text：主要是将text转换为tokens，然后进一步转换为token IDs
        token_ids = tokenizer.encode(txt, allowed_special = {"<|endoftext|>"})

        print("token_ids", token_ids)
        print("len(token_ids)", len(token_ids))
        print("max_length", max_length)

        # 使用滑动窗口将输入截断（chunk）为长度是max_length的重合序列
        if len(token_ids) > max_length:
            for i in range(0, len(token_ids) - max_length, stride):  #这里没有考虑max_
                input_chunk = token_ids[i : i + max_length] 
                target_chunk = token_ids[i + 1 : i + max_length + 1]
                self.input_ids.append(torch.tensor(input_chunk))
                self.target_ids.append(torch.tensor(target_chunk))
        else:
            self.input_ids.append(torch.tensor(token_ids[0 : 0 + max_length]))
            self.target_ids.append(torch.tensor(token_ids[0 : 0 + max_length]))

    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

Note：<br>
1.在使用sliding window采样得到输入的时候，end的值为len(token_ids) - max_length的原因是为了防止数组越界，因为如果max_length = 4,而且数据集tokenization之后一共只有5个tokens，那么就只能采样1次。上面的这个例子，没有考虑max_length >= len(token_ids)的情况。

2.这里stride控制的是 i 的间隔，也就是说滑动窗口移动的步数, 这就导致了batch中不同sample之间的起始token的间隔。

3.range()函数的用法：<br>
（1）语法：range(start, end, step)。是生成\[start, end\)之间的序列，不是一次性生成，而是iterable的。<br>
（2）不能直接输出range(1,3)，如果要输出，则应该使用list(range(1,3))<br>
（3）step可以是负数，效果是反向迭代，比如range(10, 0, -2)<br>

In [ ]:
# Create the dataloader
def create_dataloader_v1(txt, batch_size = 4, max_length = 256, stride = 128, \
                         shuffle = True, drop_last = True, num_workers = 0):
    tokenizer = tiktoken.get_encoding("gpt2")

    # 创建dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    #使用dataloader将dataset封装为一个dataloader数据加载器    
    dataloader = DataLoader(
        dataset, 
        batch_size = batch_size,
        shuffle = shuffle,
        drop_last = drop_last,
        num_workers = num_workers
    )

    return dataloader

In [17]:
with open("./data/the-verdict.txt", 'r', encoding='utf-8') as f:
    raw_text = f.read()


In [24]:
dataloader = create_dataloader_v1(
    raw_text,
    batch_size = 1,
    max_length = 4, 
    stride = 1,
    shuffle = False
)

In [ ]:
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)
# 返回的结果里面，第一个tensor是x，每一行代表一个input sample的context。、
# 第二个tensor是y，对应位置就是target sample。

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [ ]:
second_batch = next(data_iter)
print(second_batch)
# 可以看到第二个batch和第一个batch之间不同之处，就是往后移动了1个位置，
# 这是因为stride = 1。

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [ ]:
#将stride 修改为 3
dataloader = create_dataloader_v1(
    raw_text,
    batch_size = 1,
    max_length = 4, 
    stride = 3, #修改为3
    shuffle = False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

second_batch = next(data_iter)
print(second_batch)

# 可以看到修改之后，batch之间的间隔从1变成3了，但是值得注意的是，input和target之间的间隔还是始终为1
# 因为任务始终是next token prediciton.
# stride仅仅控制batch之间的位置间隔，不控制input和target之间的间隔——在batch_size = 1的时候适用；
# 如果batch_size不等于1，那么stride控制的是一个batch内部sample之间起始位置的间隔，或者说控制的是batch
# 之间的上一个batch的最后一个sample，与下一个batch的第一个sample之间，起始位置的间隔。

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[1464, 1807, 3619,  402]]), tensor([[1807, 3619,  402,  271]])]


In [31]:
# 尝试不同的参数组合
dataloader = create_dataloader_v1(
    raw_text,
    batch_size = 8,
    max_length = 4, 
    stride = 4,
    shuffle = False
)

data_iter = iter(dataloader)
first_batch_input, first_batch_target  = next(data_iter)
print(first_batch_input)
print(first_batch_target)

second_batch_input, second_batch_target  = next(data_iter)
print(second_batch_input)
print(second_batch_target)

tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
tensor([[  287,   262,  6001,   286],
        [  465, 13476,    11,   339],
        [  550,  5710,   465, 12036],
        [   11,  6405,   257,  5527],
        [27075,    11,   290,  4920],
        [ 2241,   287,   257,  4489],
        [   64,   319,   262, 34686],
        [41976,    13,   357, 10915]])
tensor([[  262,  6001,   286,   465],
        [13476,    11,   339,   550],
        [

In [43]:
# test the case that if the len(token_ids) <= max_length
raw_text = "hello world"

dataloader = create_dataloader_v1(
    raw_text,
    batch_size = 1,
    max_length = 2, # equal to len(token_ids) 
    stride = 1,
    shuffle = False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

token_ids [31373, 995]
len(token_ids) 2
max_length 2
[tensor([[31373,   995]]), tensor([[31373,   995]])]
